In [1]:
BUCKET_URI = f"gs://udemy-gcp-mlops" 
PROJECT_ID='dataanalytics-347914'
REGION = "us-central1"
gcs_source = f"{BUCKET_URI}/data_petfinder-tabular-classification.csv"

In [2]:
import os
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

In [3]:
ds = dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=gcs_source,
)

ds.resource_name

Creating TabularDataset
Create TabularDataset backing LRO: projects/936546808722/locations/us-central1/datasets/5490205154869248000/operations/5200258063109455872
TabularDataset created. Resource name: projects/936546808722/locations/us-central1/datasets/5490205154869248000
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/936546808722/locations/us-central1/datasets/5490205154869248000')


'projects/936546808722/locations/us-central1/datasets/5490205154869248000'

In [4]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-petfinder-automl-1",
    optimization_prediction_type="classification",
    column_transformations=[
        {"categorical": {"column_name": "Type"}},
        {"numeric": {"column_name": "Age"}},
        {"categorical": {"column_name": "Breed1"}},
        {"categorical": {"column_name": "Color1"}},
        {"categorical": {"column_name": "Color2"}},
        {"categorical": {"column_name": "MaturitySize"}},
        {"categorical": {"column_name": "FurLength"}},
        {"categorical": {"column_name": "Vaccinated"}},
        {"categorical": {"column_name": "Sterilized"}},
        {"categorical": {"column_name": "Health"}},
        {"numeric": {"column_name": "Fee"}},
        {"numeric": {"column_name": "PhotoAmt"}},
    ],
)

In [5]:
model = job.run(
    dataset=ds,
    target_column="Adopted",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name="adopted-prediction-model",
    disable_early_stopping=False,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6553006941252091904?project=936546808722
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/6553006941252091904 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/6553006941252091904 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/6553006941252091904 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/6553006941252091904 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/6553006941252091904 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trai

KeyboardInterrupt: 

In [5]:
MODEL_ID='8525391060426293248'
model = aiplatform.Model('projects/{}/locations/us-central1/models/{}'.format(PROJECT_ID,MODEL_ID))

In [6]:
endpoint = model.deploy(
    machine_type="n1-standard-4",
)

Creating Endpoint
Create Endpoint backing LRO: projects/1090925531874/locations/us-central1/endpoints/3738412102205833216/operations/836265110744858624
Endpoint created. Resource name: projects/1090925531874/locations/us-central1/endpoints/3738412102205833216
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1090925531874/locations/us-central1/endpoints/3738412102205833216')
Deploying model to Endpoint : projects/1090925531874/locations/us-central1/endpoints/3738412102205833216
Deploy Endpoint model backing LRO: projects/1090925531874/locations/us-central1/endpoints/3738412102205833216/operations/2822352546415247360
Endpoint model deployed. Resource name: projects/1090925531874/locations/us-central1/endpoints/3738412102205833216


In [9]:
prediction = endpoint.predict(
    [
        {
            "Type": "Cat",
            "Age": "3",
            "Breed1": "Tabby",
            "Gender": "Male",
            "Color1": "Black",
            "Color2": "White",
            "MaturitySize": "Small",
            "FurLength": "Short",
            "Vaccinated": "No",
            "Sterilized": "No",
            "Health": "Healthy",
            "Fee": "100",
            "PhotoAmt": "2",
        }
    ]
)

In [10]:
prediction

Prediction(predictions=[{'scores': [0.7372735142707825, 0.2627264857292175], 'classes': ['Yes', 'No']}], deployed_model_id='4126856366159560704', model_version_id='1', model_resource_name='projects/1090925531874/locations/us-central1/models/8525391060426293248', explanations=None)